In [9]:
# -*- coding:uft-8 -*-
'''
下载图片数据并解压
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz
下载训练好的Inception-v3模型
wget https://storage.googleapis.com/download.tensorflow.org/models/inception_dec_2015.zip
unzip -o -d inception_dec_2015  inception_dec_2015.zip
'''
import glob
import os.path
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

# Inception-v3模型瓶颈层的节点个数
BOTTLENECK_TENSOR_SIZE = 2048
# Inception-v3模型中代表瓶颈层结果的张量名称
BOTTLENECK_TENSOR_NAME = 'pool_3/_reshape:0'
# 图像输入张量所对应的名称
JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0'
# 下载的Inception-v3模型文件路径
MODEL_DIR = 'inception_dec_2015'
# 下载的Inception-v3模型文件名
MODEL_FILE = 'tensorflow_inception_graph.pb'
# 特征向量保存路径
CACHE_DIR = 'tmp/bottleneck'
# 图像数据路径
INPUT_DATA = 'flower_photos'

# 验证数据百分比
VALIDATION_PERCENTAGE = 10
# 测试数据百分比
TEST_PERCENTAGE = 10

# 配置神经网络的设置
LEARNING_RATE = 0.01
STEPS = 4000
BATCH = 100

# 划分训练数据集、测试数据集、验证数据集
def create_image_lists(testing_percentage, validation_percentage):
    result = {}# 返回划分结果字典
    # 获取指定目录下所有子目录
    sub_dirs = [x[0] for x in os.walk(INPUT_DATA)]
    # 得到的第一个目录时当前目录
    is_root_dir = True
    for sub_dir in sub_dirs:
        # 根目录无需操作
        if is_root_dir:
            is_root_dir = False
            continue
        # 获取当前目录下所有有效图片文件
        extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
        file_list = []
        dir_name = os.path.basename(sub_dir)
        for extension in extensions:
            file_glob = os.path.join(INPUT_DATA, dir_name, '*.'+extension)
            file_list.extend(glob.glob(file_glob))
        if not file_list: continue
        # 通过目录名获取类别的名称
        label_name = dir_name.lower()
        # 初始化当前类别的训练集、测试集、验证集
        training_images = []
        testing_images = []
        validation_images = []
        for file_name in file_list:
            base_name = os.path.basename(file_name)
            # 随机分配
            chance = np.random.randint(100)
            if chance < validation_percentage:
                validation_images.append(base_name)
            elif chance < (testing_percentage+validation_percentage):
                testing_images.append(base_name)
            else:
                training_images.append(base_name)
        # 将当前类别的数据放入结果字典
        result[label_name] = {
            'dir': dir_name,
            'training': training_images,
            'testing': testing_images,
            'validation': validation_images
        }
    return result

# 根据类别名称、所属数据集和图片编号获取一张图片的地址
def get_image_path(image_lists, image_dir, label_name, index, category):
    # 获取给定类别中所有图片的信息
    label_lists = image_lists[label_name]
    # 根据所属数据集的名称获取集合中的全部图片信息
    category_list = label_lists[category]
    mod_index = index%len(category_list)
    # 获取图片的文件名
    base_name = category_list[mod_index]
    sub_dir = label_lists['dir']
    # 最终的地址为数据根目录的地址加上类别的文件夹加上图片的名称
    full_path = os.path.join(image_dir, sub_dir, base_name)
    
    return full_path

# 根据类别名称、所属数据集和图片编号获取经模型处理后的特征向量地址
def get_bottleneck_path(image_lists, label_name, index, category):
    return get_image_path(image_lists, CACHE_DIR, label_name, index, category)+'.txt'

# 运行模型得到一张图片的特征向量
def run_bottleneck_on_image(sess, image_data, image_data_tensor, bottleneck_tensor):
    # 将当前图片作为输入计算瓶颈张量的值，瓶颈张量的值就是该图片的新特征向量
    bottleneck_values = sess.run(bottleneck_tensor, {image_data_tensor: image_data})
    # 由于卷积结果是四维数组，所以需要将其压缩为一维特征向量
    bottleneck_values = np.squeeze(bottleneck_values)
    
    return bottleneck_values

# 获取一张图片已经处理后的特征向量
def get_or_create_bottleneck(sess, image_lists, label_name, index, category, jpeg_data_tensor, bottleneck_tensor):
    # 获取一张图片对应的特征向量文件的路径
    label_lists = image_lists[label_name]
    sub_dir = label_lists['dir']
    sub_dir_path = os.path.join(CACHE_DIR, sub_dir)
    # 路径不存在则创建
    if not os.path.exists(sub_dir_path): os.makedirs(sub_dir_path)
    bottleneck_path = get_bottleneck_path(image_lists, label_name, index, category)
    
    # 如果特征向量文件不存在，则通过模型计算结果并存入
    if not os.path.exists(bottleneck_path):
        # 获取原始图片路径
        image_path = get_image_path(image_lists, INPUT_DATA, label_name, index, category)
        # 获取图片内容
        image_data = gfile.FastGFile(image_path, 'rb').read()
        # 通过Inception-v3模型计算特征向量
        bottleneck_values = run_bottleneck_on_image(sess, image_data, jpeg_data_tensor, bottleneck_tensor)
        # 存储特征向量
        bottleneck_string = ','.join(str(x) for x in bottleneck_values)
        with open(bottleneck_path, 'w') as bottleneck_file:
            bottleneck_file.write(bottleneck_string)
    else:
        # 直接从文件中读出特征向量
        with open(bottleneck_path, 'r') as bottleneck_file:
            bottleneck_string = bottleneck_file.read()
        bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
    return bottleneck_values

# 获取随机batch的训练集
def get_random_cached_bottleneck(sess, n_classes, image_lists, how_many, category, jpeg_data_tensor, bottleneck_tensor):
    bottlenecks = []
    ground_truths = []
    for _ in range(how_many):
        # 随机一个类别和图片编号加入当前的训练数据
        label_index = random.randrange(n_classes)
        label_name = list(image_lists.keys())[label_index]
        image_index = random.randrange(65536)
        bottleneck = get_or_create_bottleneck(sess, image_lists, label_name, image_index, category, jpeg_data_tensor, bottleneck_tensor)
        ground_truth = np.zeros(n_classes, dtype=np.float32)
        ground_truth[label_index] = 1.0
        bottlenecks.append(bottleneck)
        ground_truths.append(ground_truth)
    return bottlenecks, ground_truths

# 获取全部测试数据
def get_test_bottlenecks(sess, n_classes, image_lists, jpeg_data_tensor, bottleneck_tensor):
    bottlenecks = []
    ground_truths = []
    label_name_list = list(image_lists.keys())
    # 枚举所有的类别和每个类别中的测试图片
    for label_index, label_name in enumerate(label_name_list):
        category = 'testing'
        for index, unused_base_name in enumerate(image_lists[label_name][category]):
            # 计算特征向量，并加入最终列表
            bottleneck = get_or_create_bottleneck(sess, image_lists, label_name, index, category, jpeg_data_tensor, bottleneck_tensor)
            ground_truth = np.zeros(n_classes, dtype=np.float32)
            ground_truth[label_index] = 1.0
            bottlenecks.append(bottleneck)
            ground_truths.append(ground_truth)
    return bottlenecks, ground_truths

def main(_):
    # 读取所有图片
    image_lists = create_image_lists(TEST_PERCENTAGE, VALIDATION_PERCENTAGE)
    n_classes = len(image_lists.keys())
    # 读取已经训练好的Inception-v3模型
    with gfile.FastGFile(os.path.join(MODEL_DIR, MODEL_FILE), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    # 加载模型
    bottleneck_tensor, jpeg_data_tensor = tf.import_graph_def(graph_def, return_elements=[BOTTLENECK_TENSOR_NAME, JPEG_DATA_TENSOR_NAME])
    # 定义新的神经网络输入
    bottleneck_input = tf.placeholder(tf.float32, [None, BOTTLENECK_TENSOR_SIZE], name='BottleneckInput')
    # 定义新的标准答案输入
    ground_truth_input = tf.placeholder(tf.float32, [None, n_classes], name='GroundTruthInput')
    # 定义全连接，解决新的分类问题
    with tf.name_scope('final_training_ops'):
        weights = tf.Variable(tf.truncated_normal([BOTTLENECK_TENSOR_SIZE, n_classes], stddev=0.001))
        biases = tf.Variable(tf.zeros([n_classes]))
        logits = tf.matmul(bottleneck_input, weights)+biases
        final_tensor = tf.nn.softmax(logits)
    # 定义交叉熵
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=ground_truth_input)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy_mean)
    # 计算正确率
    with tf.name_scope('evaluation'):
        correct_prediction = tf.equal(tf.argmax(final_tensor, 1), tf.argmax(ground_truth_input,1))
        evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 训练
        for i in range(STEPS):
            train_bottlenecks, train_ground_truth = get_random_cached_bottleneck(sess, n_classes, image_lists, BATCH, 'training', jpeg_data_tensor, bottleneck_tensor)
            sess.run(train_step, feed_dict={bottleneck_input:train_bottlenecks, ground_truth_input:train_ground_truth})
            
            # 正确率-验证数据集
            if i%100 == 0 or i+1 == STEPS:
                validation_bottlenecks, validation_ground_truth = get_random_cached_bottleneck(sess, n_classes, image_lists, BATCH, 'validation', jpeg_data_tensor, bottleneck_tensor)
                validation_accuracy = sess.run(evaluation_step, feed_dict={bottleneck_input:validation_bottlenecks, ground_truth_input:validation_ground_truth})
                print('step %d: validation accuracy on random sampled %d examples = %.1f%%'%(i,BATCH,validation_accuracy*100))
            
        # 正确率-测试数据集
        test_bottlenecks, test_ground_truth = get_test_bottlenecks(sess, n_classes, image_lists, jpeg_data_tensor, bottleneck_tensor)
        test_accuracy = sess.run(evaluation_step, feed_dict={bottleneck_input:test_bottlenecks, ground_truth_input:test_ground_truth})
        print('final test accuracy = %.1f%%'%(test_accuracy*100))

if __name__ == '__main__':
    tf.app.run()

step 0: validation accuracy on random sampled 100 examples = 29.0%
step 100: validation accuracy on random sampled 100 examples = 81.0%
step 200: validation accuracy on random sampled 100 examples = 90.0%
step 300: validation accuracy on random sampled 100 examples = 90.0%
step 400: validation accuracy on random sampled 100 examples = 93.0%
step 500: validation accuracy on random sampled 100 examples = 88.0%
step 600: validation accuracy on random sampled 100 examples = 84.0%
step 700: validation accuracy on random sampled 100 examples = 87.0%
step 800: validation accuracy on random sampled 100 examples = 89.0%
step 900: validation accuracy on random sampled 100 examples = 89.0%
step 1000: validation accuracy on random sampled 100 examples = 90.0%
step 1100: validation accuracy on random sampled 100 examples = 98.0%
step 1200: validation accuracy on random sampled 100 examples = 86.0%
step 1300: validation accuracy on random sampled 100 examples = 93.0%
step 1400: validation accuracy o

NameError: name 'tets_ground_truth' is not defined